In [ ]:
!pip install torch-geometric -q

In [ ]:
import os
import gc
import torch
import numpy as np
import pandas as pd
import random
import itertools
import time
from collections import defaultdict
from tqdm import tqdm
from torch_geometric.nn import LightGCN
from torch_geometric.utils import remove_self_loops, to_undirected

# ===============================================================
# 0. CONFIGURACIÓN OPTIMIZADA (40GB VRAM)
# ===============================================================
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(SEED)

K_METRIC = 20
SAVE_DIR = "results_lightgcn_40gb"
os.makedirs(SAVE_DIR, exist_ok=True)

# ===============================================================
# 1. PREPARACIÓN DE DATOS Y POPULARIDAD
# ===============================================================
def preprocess_data(df_pairs):
    print("🔍 Filtrando autores con >= 2 colaboraciones...")
    counts = pd.concat([df_pairs['pair_min'], df_pairs['pair_max']]).astype(str).value_counts()
    eligible_authors = counts[counts >= 2].index
    df_filtered = df_pairs[
        df_pairs['pair_min'].astype(str).isin(eligible_authors) &
        df_pairs['pair_max'].astype(str).isin(eligible_authors)
    ].copy()
    return df_filtered

def get_item_popularity_tensor(df_train, author_to_idx, n_authors):
    pop_counts = pd.concat([df_train['pair_min'], df_train['pair_max']]).astype(str).value_counts()
    total_interactions = len(df_train) * 2
    popularity_array = np.ones(n_authors)
    for auth_id, count in pop_counts.items():
        if auth_id in author_to_idx:
            popularity_array[author_to_idx[auth_id]] = count
    novelty_scores = -np.log2(popularity_array / total_interactions)
    return torch.tensor(novelty_scores, dtype=torch.float, device=DEVICE)

def triple_loo_split(df, seed=42):
    print("✂️ Generando Split LOO...")
    rng = np.random.default_rng(seed)
    adj = defaultdict(list)
    for idx, row in enumerate(df.itertuples()):
        u, v = str(row.pair_min), str(row.pair_max)
        adj[u].append(idx); adj[v].append(idx)

    test_idx, val_idx, assigned = set(), set(), set()
    for author in adj:
        indices = [i for i in adj[author] if i not in assigned]
        if len(indices) >= 3:
            rng.shuffle(indices)
            test_idx.add(indices[0]); val_idx.add(indices[1])
            assigned.update([indices[0], indices[1]])
        elif len(indices) == 2:
            t = rng.choice(indices)
            test_idx.add(t); assigned.add(t)

    all_indices = np.arange(len(df))
    train_idx = np.setdiff1d(all_indices, list(test_idx | val_idx))
    return df.iloc[train_idx], df.iloc[list(val_idx)], df.iloc[list(test_idx)]

# ===============================================================
# 2. UTILIDADES DE GRAFO
# ===============================================================
def build_edge_index(df, mapping):
    u = df["pair_min"].astype(str).map(mapping).values
    v = df["pair_max"].astype(str).map(mapping).values
    edge_index = torch.tensor(np.array([u, v]), dtype=torch.long)
    edge_index = to_undirected(edge_index)
    edge_index, _ = remove_self_loops(edge_index)
    return edge_index.to(DEVICE)

def build_adj_dict(df, mapping):
    adj = defaultdict(set)
    for row in df.itertuples():
        u, v = mapping[str(row.pair_min)], mapping[str(row.pair_max)]
        adj[u].add(v); adj[v].add(u)
    return adj

def sample_batch_fast(adj_list, keys, num_nodes, batch_size):
    u = np.random.choice(keys, size=batch_size, replace=True)
    p = np.array([random.choice(adj_list[user]) for user in u])
    n = np.random.randint(0, num_nodes, size=batch_size)
    return (torch.tensor(u, device=DEVICE), torch.tensor(p, device=DEVICE), torch.tensor(n, device=DEVICE))

# ===============================================================
# 3. EVALUACIÓN OPTIMIZADA PARA 40GB VRAM
# ===============================================================
@torch.inference_mode()
def calculate_metrics_gpu(model, edge_index, R_train_adj, R_target_adj, users_to_eval, novelty_tensor, K=20):
    model.eval()
    # Usamos float16 para ahorrar memoria en la matriz de scores
    emb = model.get_embedding(edge_index).half()

    # eval_batch_size reducido de 20k a 4k para evitar OOM en 40GB
    eval_batch_size = 4000
    recall_list, ndcg_list, novelty_list = [], [], []
    recommended_items_all = set()

    for i in range(0, len(users_to_eval), eval_batch_size):
        batch_u = torch.tensor(users_to_eval[i:i + eval_batch_size], device=DEVICE)

        # Matriz de scores en media precisión
        scores = torch.matmul(emb[batch_u], emb.t())

        # Filtrado de ítems vistos (masking)
        for idx, u_idx in enumerate(batch_u.cpu().numpy()):
            seen = list(R_train_adj.get(u_idx, set()))
            if seen: scores[idx, seen] = -1e4 # En float16 evitamos -1e10 para no desbordar
            scores[idx, u_idx] = -1e4

        _, top_indices = torch.topk(scores, K, dim=1)

        # Cálculo de Novelty
        batch_nov = novelty_tensor[top_indices].mean(dim=1).cpu().numpy()
        novelty_list.extend(batch_nov)

        top_indices_cpu = top_indices.cpu().numpy()
        recommended_items_all.update(top_indices_cpu.flatten())

        for idx, u_idx in enumerate(batch_u.cpu().numpy()):
            targets = list(R_target_adj.get(u_idx, set()))
            if not targets: continue
            preds = top_indices_cpu[idx]
            hits = np.isin(preds, targets)
            hit_count = np.sum(hits)

            recall_list.append(hit_count / len(targets))
            if hit_count > 0:
                rank_pos = np.where(hits)[0] + 2
                dcg = np.sum(1.0 / np.log2(rank_pos))
                idcg = np.sum(1.0 / np.log2(np.arange(min(len(targets), K)) + 2))
                ndcg_list.append(dcg / idcg)
            else:
                ndcg_list.append(0.0)

        # Liberación explícita de memoria del batch
        del scores, batch_u, top_indices
        torch.cuda.empty_cache()

    return {
        "recall": np.mean(recall_list) if recall_list else 0,
        "ndcg": np.mean(ndcg_list) if ndcg_list else 0,
        "novelty": np.mean(novelty_list) if novelty_list else 0,
        "coverage": len(recommended_items_all) / model.num_nodes
    }

# ===============================================================
# 4. EXPERIMENTO PRINCIPAL
# ===============================================================
def run_optimized_experiment(df_raw, author_to_idx_raw):
    author_to_idx = {str(k): int(v) for k, v in author_to_idx_raw.items()}
    n_authors = len(author_to_idx)

    df_filtered = preprocess_data(df_raw)
    df_train, df_val, df_test = triple_loo_split(df_filtered)

    edge_index_train = build_edge_index(df_train, author_to_idx)
    adj_train = build_adj_dict(df_train, author_to_idx)
    adj_val = build_adj_dict(df_val, author_to_idx)

    train_keys = list(adj_train.keys())
    adj_train_list = {k: list(v) for k, v in adj_train.items()}
    novelty_train = get_item_popularity_tensor(df_train, author_to_idx, n_authors)

    configs = [
            {
                'dim': 256,
                'layers': 2,
                'lr': 5e-4,
                'reg': 1e-2,
                'name': 'Anchor-Heavy'
            },
            {
                'dim': 256,
                'layers': 1,      # Solo vecinos directos para evitar ruido
                'lr': 1e-3,
                'reg': 1e-3,
                'name': 'Shallow-Local'
            },
            {
                'dim': 512,
                'layers': 2,
                'lr': 5e-4,
                'reg': 1e-1,      # Regularización extrema para "matar" el ruido
                'name': 'Extreme-Reg'
            }
        ]
    val_users = list(adj_val.keys())
    sample_val = np.random.choice(val_users, min(20000, len(val_users)), replace=False)
    best_ndcg, best_params = -1, {}

    # Mantenemos el BATCH_SIZE de entrenamiento (131k) porque el entrenamiento
    # consume mucha menos VRAM que la evaluación matricial completa.
    BATCH_SIZE = 131072

    for i, conf in enumerate(configs):
        print(f"\n🚀 Iniciando Config {i+1} [{conf['name']}]: Dim={conf['dim']}, Layers={conf['layers']}")
        model = LightGCN(num_nodes=n_authors, embedding_dim=conf['dim'], num_layers=conf['layers']).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=conf['lr'])

        for epoch in range(1, 61):
            model.train()
            u, p, n = sample_batch_fast(adj_train_list, train_keys, n_authors, BATCH_SIZE)
            optimizer.zero_grad()
            emb = model.get_embedding(edge_index_train)
            loss = model.recommendation_loss((emb[u]*emb[p]).sum(dim=1), (emb[u]*emb[n]).sum(dim=1), torch.cat([u,p,n]).unique(), lambda_reg=conf['reg'])
            loss.backward()
            optimizer.step()

        m = calculate_metrics_gpu(model, edge_index_train, adj_train, adj_val, sample_val, novelty_train)
        print(f"  📊 Val Final -> Recall: {m['recall']:.4f} | NDCG: {m['ndcg']:.4f} | Novelty: {m['novelty']:.4f}")

        if m['ndcg'] > best_ndcg:
            best_ndcg = m['ndcg']; best_params = conf

        del model
        torch.cuda.empty_cache()
        gc.collect()

    # --- ENTRENAMIENTO FINAL ---
    print(f"\n🏆 GANADOR: {best_params['name']} con NDCG: {best_ndcg:.4f}")
    df_full_train = pd.concat([df_train, df_val])
    edge_index_full = build_edge_index(df_full_train, author_to_idx)
    adj_full = build_adj_dict(df_full_train, author_to_idx)
    adj_full_list = {k: list(v) for k, v in adj_full.items()}
    full_keys = list(adj_full.keys())
    novelty_full = get_item_popularity_tensor(df_full_train, author_to_idx, n_authors)

    final_model = LightGCN(num_nodes=n_authors, embedding_dim=best_params['dim'], num_layers=best_params['layers']).to(DEVICE)
    optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params['lr'])

    for epoch in tqdm(range(1, 201), desc="Training Final Model"):
        final_model.train()
        u, p, n = sample_batch_fast(adj_full_list, full_keys, n_authors, BATCH_SIZE)
        optimizer.zero_grad()
        emb = final_model.get_embedding(edge_index_full)
        loss = final_model.recommendation_loss((emb[u]*emb[p]).sum(dim=1), (emb[u]*emb[n]).sum(dim=1), torch.cat([u,p,n]).unique(), lambda_reg=best_params['reg'])
        loss.backward()
        optimizer.step()

    # --- EVALUACIÓN TEST ---
    adj_test = build_adj_dict(df_test, author_to_idx)
    test_users = list(adj_test.keys())
    res = calculate_metrics_gpu(final_model, edge_index_full, adj_full, adj_test, test_users, novelty_full)

    print("\n" + "="*50)
    print("🎯 RESULTADO FINAL LIGHTGCN EN TEST")
    print(f"Recall@20: {res['recall']:.6f}")
    print(f"NDCG@20:   {res['ndcg']:.6f}")
    print(f"Novelty:   {res['novelty']:.6f}")
    print(f"Coverage:  {res['coverage']:.2%}")
    print("="*50)

    torch.save(final_model.state_dict(), f"{SAVE_DIR}/lightgcn_final.pt")

if __name__ == "__main__":
    df_raw = pd.read_pickle("data/df_pairs_unique.pkl")
    author_to_idx = np.load("data/author_to_idx.npy", allow_pickle=True).item()
    run_optimized_experiment(df_raw, author_to_idx)
